In [1]:
import tensorflow as tf

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)
        
def read_my_file_format(filename_queue):
    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(filename_queue)
    
    record_defaults = [[1] for i in range(541)]
#     print(record_defaults)

    cols = tf.decode_csv(value, record_defaults=record_defaults)
    labels = tf.one_hot(cols[1], 2)
    features = tf.stack(cols[2:])

    return features, labels

def input_pipeline(filenames, batch_size, num_epochs=None):
    filename_queue = tf.train.string_input_producer(
        filenames, num_epochs=num_epochs, shuffle=True)
    example, label = read_my_file_format(filename_queue)
  # min_after_dequeue defines how big a buffer we will randomly sample
  #   from -- bigger means better shuffling but slower start up and more
  #   memory used.
  # capacity must be larger than min_after_dequeue and the amount larger
  #   determines the maximum we will prefetch.  Recommendation:
  #   min_after_dequeue + (num_threads + a small safety margin) * batch_size
    min_after_dequeue = 10000
    capacity = min_after_dequeue + 3 * batch_size
    example_batch, label_batch = tf.train.shuffle_batch(
        [example, label], batch_size=batch_size, capacity=capacity,
        min_after_dequeue=min_after_dequeue)
    return example_batch, label_batch

with tf.name_scope('CSV_data'):
    with tf.name_scope('train_approved'):
        train_features_app, train_labels_app = input_pipeline(["data/train_approved.csv"], 200)
    with tf.name_scope('train_rejected'):
        train_features_rej, train_labels_rej = input_pipeline(["data/train_rejected.csv"], 100)
    with tf.name_scope('train_features'):
        train_features = tf.concat([train_features_app, train_features_rej], 0)
        print("train_features shape: " + str(train_features))
    with tf.name_scope('train_labels'):
        train_labels = tf.concat([train_labels_app, train_labels_rej], 0)
        print("train_labels shape: " + str(train_labels))

# train_features, train_labels = input_pipeline(["data/train_approved.csv", "data/train_rejected.csv"], 100)

    with tf.name_scope('test_data'):
        test_features, test_labels = input_pipeline(["data/test.csv"], 100)


init = tf.global_variables_initializer()

# with tf.Session() as sess:
#     sess.run(init)
    
#     coord = tf.train.Coordinator()
#     threads = tf.train.start_queue_runners(coord=coord)

#     print( sess.run([train_features, train_labels]) )
    
#     coord.request_stop()
#     coord.join(threads)
    
# print("\nEnd.\n")


train_features shape: Tensor("CSV_data/train_features/concat:0", shape=(300, 539), dtype=int32)
train_labels shape: Tensor("CSV_data/train_labels/concat:0", shape=(300, 2), dtype=float32)


In [2]:

#define place holder
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 539])
    y = tf.placeholder(tf.float32, [None, 2])


#create a 3-level neural network
with tf.name_scope('Layers'):
    with tf.name_scope('Layers1'):
        dense1 = tf.layers.dense(inputs=x, units=300, activation=tf.nn.relu)
#         variable_summaries(dense1)

    with tf.name_scope('Layers2'):
        prediction = tf.layers.dense(inputs=dense1, units=2, activation=tf.nn.softmax)
#         variable_summaries(prediction)

with tf.name_scope('train'):
    with tf.name_scope('learning_rate'):
        #学习率使用指数衰减法
        global_step = tf.Variable(0)
        learn_rate = tf.train.exponential_decay(0.1, global_step, 100, 0.95, staircase=True)
        tf.summary.scalar('learning_rate', learn_rate)
        
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
        # loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=prediction))
        tf.summary.scalar('loss', loss)
        
    with tf.name_scope('train'):
        #train
        train = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    with tf.name_scope('Init'):
        init = tf.global_variables_initializer()

    with tf.name_scope('accuracy'):
        #比较预测值与实际值
        correct_pred = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1)) #argmax 返回沿着某个维度最大值的位置

        #准确率
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        tf.summary.scalar('accuracy', accuracy)

with tf.name_scope('Summary'):
    with tf.name_scope('train_sum'):
        y_sum = tf.reduce_sum(y, 0)
        variable_summaries(y_sum)

    with tf.name_scope('prediction_sum'):
        # pred_one_hot = tf.stack(tf.argmin(prediction, 1), tf.argmax(prediction, 1))
        # a = tf.argmin(prediction, 1)
        # b = tf.argmax(prediction, 1)
        # pred_one_hot = tf.transpose(tf.stack([a, b]))
        pred_one_hot = tf.one_hot(tf.argmax(prediction, 1), 2)
        # pred_one_hot = tf.one_hot(tf.argmax(prediction, 1), tf.shape(prediction)[0])
#         print(pred_one_hot)

        pred_sum = tf.reduce_sum(pred_one_hot, 0)
    
    with tf.name_scope('intersection_sum'):
        pred_int_one_hot = tf.one_hot(tf.argmax(prediction, 1), 2, off_value=2.0)
        inter = tf.equal(y, pred_int_one_hot)
#         print(inter)

        int_sum = tf.reduce_sum(tf.cast(inter, tf.int32), 0)

        # correct_rej = tf.equal(tf.argmax(y, 1), tf.argmax(pred_one_hot, 1))
        # int_rej_num = tf.reduce_sum(tf.cast(correct_rej, tf.int32))

        # correct_app = tf.equal(tf.argmin(y, 1), tf.argmin(pred_one_hot, 1))
        # int_app_num = tf.reduce_sum(tf.cast(correct_app, tf.int32))

merged = tf.summary.merge_all()


In [3]:

with tf.Session() as sess:
    sess.run(init)
    
    writer = tf.summary.FileWriter('logs/', sess.graph)
    
    # Start populating the filename queue.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    for step in range(121):
        # Retrieve a single instance:
        batch_xs, batch_ys = sess.run([train_features, train_labels])
#         print(batch_xs.shape, batch_ys.shape)
#         print(batch_xs)
    
        summary, _ = sess.run([merged, train], feed_dict={x: batch_xs, y: batch_ys})
#         sess.run(train, feed_dict={x: batch_xs, y: batch_ys})

        if step % 10 == 0:
            writer.add_summary(summary, step)
            
            acc_train = sess.run(accuracy, feed_dict={x:batch_xs, y:batch_ys})
            
            test_xs, test_ys = sess.run([test_features, test_labels])
            pred, acc_test = sess.run([prediction, accuracy], feed_dict={x:test_xs, y:test_ys})
            print("step:" + str(step) + " train accurancy:" + str(acc_train) + "  test accurancy:" + str(acc_test))
            
            y_s, p_s, i_s = sess.run([y_sum, pred_sum, int_sum], feed_dict={x:batch_xs, y:batch_ys} )
            print("Train - Label sum:" + str(y_s) + " Prediction sum" + str(p_s) + " Correct sum:" + str(i_s))
        
            y_s, p_s, i_s = sess.run([y_sum, pred_sum, int_sum], feed_dict={x:test_xs, y:test_ys} )
            print("Test - Label sum:" + str(y_s) + " Prediction sum" + str(p_s) + " Correct sum:" + str(i_s))
            print("")
                  
    coord.request_stop()
    coord.join(threads)

step:0 train accurancy:0.47  test accurancy:0.34
Train - Label sum:[200. 100.] Prediction sum[109. 191.] Correct sum:[75 66]
Test - Label sum:[89. 11.] Prediction sum[25. 75.] Correct sum:[24 10]

step:10 train accurancy:0.67333335  test accurancy:0.83
Train - Label sum:[200. 100.] Prediction sum[292.   8.] Correct sum:[197   5]
Test - Label sum:[90. 10.] Prediction sum[93.  7.] Correct sum:[83  0]

step:20 train accurancy:0.6666667  test accurancy:0.93
Train - Label sum:[200. 100.] Prediction sum[300.   0.] Correct sum:[200   0]
Test - Label sum:[93.  7.] Prediction sum[100.   0.] Correct sum:[93  0]

step:30 train accurancy:0.6666667  test accurancy:0.93
Train - Label sum:[200. 100.] Prediction sum[300.   0.] Correct sum:[200   0]
Test - Label sum:[93.  7.] Prediction sum[100.   0.] Correct sum:[93  0]

step:40 train accurancy:0.6666667  test accurancy:0.96
Train - Label sum:[200. 100.] Prediction sum[300.   0.] Correct sum:[200   0]
Test - Label sum:[96.  4.] Prediction sum[100.   0